<a href="https://colab.research.google.com/github/Saiteja4555/NLP/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import atexit

import shutil

from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration



# Step 1: Load the pre-trained BlenderBot model and tokenizer

model_name = "facebook/blenderbot-1B-distill"

tokenizer = BlenderbotTokenizer.from_pretrained(model_name)

model = BlenderbotForConditionalGeneration.from_pretrained(model_name)


# Step 2: Define a function to interact with the chatbot

def interact_with_chatbot(user_input, conversation_history):

    # Step 2.1: Add user input to the conversation history

    conversation_history.append(f"User: {user_input}")

    # Step 2.2: Prepare the input text for the model

    conversation_text = " ".join(conversation_history[-5:])  # Use only the last 5 exchanges to keep context manageable

    # Step 2.3: Generate a response using the chatbot pipeline

    inputs = tokenizer(conversation_text, return_tensors="pt", truncation=True)

    response_ids = model.generate(**inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    response_text = tokenizer.decode(response_ids[0], skip_special_tokens=True)

    # Step 2.4: Add the generated response to the conversation history

    conversation_history.append(f"Chatbot: {response_text}")


# Step 3: Define a function to delete the model files from the cache directory

def delete_model_files():

    cache_dir = os.path.expanduser("~/.cache/huggingface/hub/models--facebook--blenderbot-1B-distill")

    if os.path.exists(cache_dir):

        user_input = input("Do you want to delete the model files from the cache directory? (y/n): ")

        if user_input.lower() == 'y':

            shutil.rmtree(cache_dir)

            print(f"Deleted model files from cache directory: {cache_dir}")

        else:

            print("Model files not deleted from cache directory.")

    else:

        print(f"Model files not found in cache directory: {cache_dir}")


# Step 4: Register the delete_model_files function to be called on program exit

atexit.register(delete_model_files)

# Step 5: Start the conversation loop

print("Welcome to the Indian Tourism Chatbot!")

print("Type 'quit' to end the conversation.\n")

conversation_history = []

while True:

    # Step 5.1: Get user input

    user_input = input("User: ")

    # Step 5.2: Check if the user wants to quit

    if user_input.lower() == 'quit':

        print("Thank you for using the Indian Tourism Chatbot. Goodbye!")

        break

    # Step 5.3: Generate and print the chatbot's response

    response = interact_with_chatbot(user_input, conversation_history)

    print(f"Chatbot: {response}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Welcome to the Indian Tourism Chatbot!
Type 'quit' to end the conversation.

Chatbot: None
User: quit
Thank you for using the Indian Tourism Chatbot. Goodbye!


In [2]:
# Import necessary modules

import nltk
import re
from nltk.chat.util import Chat, reflections

In [3]:
# Download NLTK data

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# Define patterns and responses

pairs = [
    [r"my name is (.*)", ["Hello %1, how can I assist you today?",]],
    [r"hi|hey|hello", ["Hello, how can I help you?", "Hey there! What can I do for you?",
                       "Hi! How can I assist you today?"]],
    [r"what is your name?", ["I am a chatbot created to assist you. You can call me Chatbot.",]],
    [r"how are you?", ["I'm a bot, so I don't have feelings, but I'm here to help you!",]],
    [r"can you help me with (.*)", ["Sure, I can help you with %1. Please provide more details.",]],
    [r"sorry (.*)", ["It's okay. How can I assist you?",]],
    [r"thank you|thanks", ["You're welcome!", "No problem!", "Happy to help!"]],
    [r"quit", ["Bye! Have a great day!", "Goodbye!"]],
    [r"(.*)", ["I'm sorry, I don't understand that. Can you rephrase?",
               "Could you please elaborate on that?"]]
]